# Installing Dependencies

In [1]:
!pip3 install --upgrade pymupdf
%cd /content

# PDFtoIMG

In [ ]:
import fitz
import os

# Open the input PDF
input_pdf = "Input.pdf"
doc = fitz.open(input_pdf)

# Define an output PDF with the same dimensions
outpdf = fitz.open()

# Loop through all the pages in the input PDF
for page_num in range(len(doc)):
    page = doc[page_num]

    # Extract existing drawings
    paths = page.get_drawings()

    # Create a new page in the output PDF
    outpage = outpdf.new_page(width=page.rect.width, height=page.rect.height)
    shape = outpage.new_shape()

    # Loop through the paths and draw them on the new page
    for path in paths:
        for item in path["items"]:
            if item[0] == "l":  # line
                shape.draw_line(item[1], item[2])
            elif item[0] == "re":  # rectangle
                shape.draw_rect(item[1])
            elif item[0] == "qu":  # quad
                shape.draw_quad(item[1])
            elif item[0] == "c":  # curve
                shape.draw_bezier(item[1], item[2], item[3], item[4])
            else:
                raise ValueError("Unhandled drawing type", item)

        # Finish the path and commit the shape to the page
        shape.finish()

    shape.commit()

# Save the output PDF
output_pdf = "line_extracted1.pdf"
outpdf.save(output_pdf)

# Create the output folder for images if it doesn't exist
output_folder = "Page"
os.makedirs(output_folder, exist_ok=True)

# Open the output PDF and save each page as an image
doc = fitz.open(output_pdf)
mat = fitz.Matrix(5.0, 5.0)

for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    pix = page.get_pixmap(matrix=mat)
    output_filename = f"page_{page_num + 1}.png"
    output_path = os.path.join(output_folder, output_filename)
    pix.save(output_path)
    print(f"Extracted image saved at: {output_path}")

print(f"All extracted images are saved in the '{output_folder}' folder.")


# Cropping Columns


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image using OpenCV
img_path = "BBS_1_image_1.png"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
# blurred_img = cv2.GaussianBlur(img, (5, 5), 0)

# Apply Canny edge detection
edges = cv2.Canny(img, 50, 150, apertureSize=3)

# Find contours using hierarchical contouring
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Detect lines using probabilistic Hough transform
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

# Draw lines on a copy of the original image
lines_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
vertical_lines = []
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if abs(x1 - x2) < abs(y1 - y2):
            cv2.line(lines_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            vertical_lines.append(int((x1 + x2) / 2))

# Sort vertical line x-coordinates
vertical_lines.sort()

# Function to filter out contours that are too close to vertical lines
def filter_contours(contours, x1, x2, margin=5):
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x1 + margin < x < x2 - margin:
            filtered_contours.append(contour)
    return filtered_contours

# Variable to store the column with the most contours
max_contours = 0
best_column_img = None
best_column_contours = None
best_x1 = 0
best_x2 = 0

# Iterate over pairs of vertical lines to find contours between them
for i in range(len(vertical_lines) - 1):
    x1 = vertical_lines[i]
    x2 = vertical_lines[i + 1]

    # Filter contours within the current column
    column_contours = filter_contours(contours, x1, x2)

    # If this column has more contours, update the best column
    if len(column_contours) > max_contours:
        max_contours = len(column_contours)
        best_column_img = img[:, x1:x2]
        best_column_contours = column_contours
        best_x1 = x1
        best_x2 = x2

# Plot the images side by side
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original image with detected vertical lines
axes[0].imshow(lines_img)
axes[0].set_title('Vertical Lines')
axes[0].axis('off')

if best_column_img is not None:
    # Cropped column with contours
    axes[1].imshow(best_column_img, cmap='gray')
    axes[1].set_title('Cropped Column')
    axes[1].axis('off')

    # Column with drawn contours
    best_column_contour_img = cv2.cvtColor(best_column_img, cv2.COLOR_GRAY2BGR)
    for contour in best_column_contours:
        cv2.drawContours(best_column_contour_img, [contour], -1, (0, 255, 0), 2)

    axes[2].imshow(best_column_contour_img)
    axes[2].set_title('Column with Contours')
    axes[2].axis('off')
    cv2.imwrite(f"cropped_column.png", best_column_img)

plt.show()

# Cropping Rows

In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def is_image_blank(image, blank_threshold=0.98, min_non_blank_pixels=100):
    # Calculate the percentage of white pixels in the image
    white_pixels = np.sum(image >= 200)  # Assuming white pixels have intensity >= 200
    total_pixels = image.size
    white_ratio = white_pixels / total_pixels

    # Check if the image has a large enough percentage of white pixels to be considered blank
    if white_ratio > blank_threshold:
        # Check if there are enough non-blank pixels to avoid mistakenly removing data rows
        non_blank_pixels = total_pixels - white_pixels
        if non_blank_pixels < min_non_blank_pixels:
            return True  # Image is considered blank
    return False  # Image is not blank

# Read the cropped column image
cropped_column_img = cv2.imread("cropped_column.png", cv2.IMREAD_GRAYSCALE)
 
# # Apply Gaussian blur to reduce noise
# blurred_img = cv2.GaussianBlur(cropped_column_img, (5, 5), 0)

# Apply adaptive thresholding to binarize the image
binary_image_adaptive = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 5)

# Apply morphological closing to connect nearby edges and smoothen the lines
kernel = np.ones((5, 5), np.uint8)
closed_img = cv2.morphologyEx(binary_image_adaptive, cv2.MORPH_CLOSE, kernel)

# Find horizontal lines using morphological operations
horizontal_lines = cv2.morphologyEx(closed_img, cv2.MORPH_OPEN, kernel, iterations=2)

# Find contours of horizontal lines
contours, _ = cv2.findContours(horizontal_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter horizontal lines based on length and horizontal coverage
filtered_lines = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w > 0.9 * cropped_column_img.shape[1]:  # Check if line spans entire width
        filtered_lines.append(contour)

# Sort filtered lines by y-coordinate
filtered_lines.sort(key=lambda c: cv2.boundingRect(c)[1])

# Define the output folder
output_folder = "Shapes"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to crop and save rows
def crop_and_save_rows(lines):
    for i in range(len(lines) - 1):
        x1, y1, w1, h1 = cv2.boundingRect(lines[i])
        x2, y2, w2, h2 = cv2.boundingRect(lines[i + 1])

        # Crop the region between two horizontal lines
        row_img = cropped_column_img[y1 + h1 : y2, :]

        # Check if the cropped row has sufficient width and is not blank
        if row_img.shape[1] > 5 and not is_image_blank(row_img):
            # Save the cropped row image
            cv2.imwrite(os.path.join(output_folder, f"row_{i + 1}.png"), row_img)

# Crop and save rows
crop_and_save_rows(filtered_lines)

# Print the number of saved rows
num_rows_saved = len(os.listdir(output_folder))
print(f"Number of saved rows: {num_rows_saved}")

# Draw the filtered horizontal lines on a copy of the original image
lines_img = cv2.cvtColor(cropped_column_img, cv2.COLOR_GRAY2BGR)
for line in filtered_lines:
    x, y, w, h = cv2.boundingRect(line)
    cv2.rectangle(lines_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the image with filtered horizontal lines
plt.figure(figsize=(10, 10))
plt.imshow(lines_img)
plt.axis('off')
plt.show()


# Bend Detection

In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import DBSCAN

def preprocess_image(image, kernel_size=(3, 3), block_size=31, C=10):
    """Preprocess image by applying Gaussian blur and adaptive thresholding."""
    binary_image = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, C
    )
    return binary_image

def find_contours(binary_image):
    """Find contours in the binary image."""
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def detect_corners_from_polygon(contour):
    """Detect corners by approximating the polygon and marking its vertices."""
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    corners = approx.reshape(-1, 2)
    return corners

def cluster_corners(corners, eps=10, min_samples=1):
    """Cluster nearby corners to ensure only one corner per vertex."""
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(corners)
    unique_corners = []
    for label in np.unique(db.labels_):
        cluster_points = corners[db.labels_ == label]
        centroid = np.mean(cluster_points, axis=0).astype(int)
        unique_corners.append(centroid)
    return np.array(unique_corners)

def detect_overlaps(image, corners, area_size=25, threshold=0.5):
    """Detect overlaps around corners based on line-like segments."""
    overlaps = []
    for corner in corners:
        x, y = corner
        area = image[max(0, y - area_size):min(image.shape[0], y + area_size),
                     max(0, x - area_size):min(image.shape[1], x + area_size)]

        # Check for line-like segments (protrusions) around the corner
        # Adjusted approach: Count black pixels in the area and threshold based on density
        black_pixel_count = np.sum(area == 0)  # Count black pixels
        area_size_squared = (2 * area_size) ** 2
        black_pixel_density = black_pixel_count / area_size_squared

        if black_pixel_density > threshold:  # Adjust threshold as needed
            overlaps.append((x, y))

    return overlaps

def is_contour_closed(contour, epsilon_factor=0.01):
    """Check if a contour is closed."""
    epsilon = epsilon_factor * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    return cv2.isContourConvex(approx)

def process_image(image_path, preprocess_params={}):
    """Process a single image and display results."""
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to read image: {image_path}")
            return

        binary_image = preprocess_image(image, **preprocess_params)
        contours = find_contours(binary_image)

        img_height, img_width = binary_image.shape
        filtered_contours = [cnt for cnt in contours if cv2.boundingRect(cnt)[2:4] != (img_width, img_height)]

        contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(contour_image, filtered_contours, -1, (0, 255, 0), thickness=2)

        plt.figure(figsize=(8, 8))
        plt.imshow(contour_image)
        plt.title('Contours Highlighted')
        plt.axis('off')
        plt.show()

        for contour in filtered_contours:
            if is_contour_closed(contour):
                corners = detect_corners_from_polygon(contour)
                corners = cluster_corners(corners)
                overlaps = detect_overlaps(binary_image, corners)
                bends = len(corners) + len(overlaps)

                plt.figure(figsize=(10, 10))
                plt.imshow(image, cmap='gray')
                plt.title('Closed Shape with Overlaps')
                plt.axis('off')

                for corner in corners:
                    x, y = corner
                    plt.plot(x, y, 'mo' if (x, y) in overlaps else 'bo', ms=10)

                for x, y in overlaps:
                    if (x, y) not in corners:
                        plt.plot(x, y, 'ro', ms=10)

                plt.show()

                print(f"Image: {image_path}")
                print(f"Closed Shape: Yes")
                print(f"Number of corners: {len(corners)}")
                print(f"Number of overlaps: {len(overlaps)}")
                print(f"Number of bends: {bends}\n")

                if len(overlaps) >= len(corners):
                    print("This case is not ideal.\n")
                break

            else:
                corners = detect_corners_from_polygon(contour)
                corners = cluster_corners(corners)
                bends = len(corners) - 2

                plt.figure(figsize=(10, 10))
                plt.imshow(image, cmap='gray')
                plt.title('Non-Closed Shape')
                plt.axis('off')

                for corner in corners:
                    plt.plot(corner[0], corner[1], 'bo', ms=10)

                plt.show()

                print(f"Image: {image_path}")
                print(f"Closed Shape: No")
                print(f"Number of corners: {len(corners)}")
                print(f"Number of bends: {bends}\n")

                break

    except Exception as e:
        print(f"An error occurred while processing '{image_path}': {e}")

# Folder path containing the images
image_folder = "Shapes"

# Check if the folder exists
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Folder '{image_folder}' does not exist.")

# Get all valid image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# If no image files found, exit early
if not image_files:
    print("No valid image files found in the folder.")
else:
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        process_image(image_path)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import DBSCAN

def add_white_border(image, border_thickness=5):
    """Add a white border by changing pixels around the edges of the image."""
    bordered_image = np.copy(image)
    bordered_image[:border_thickness, :] = 255  # Top border
    bordered_image[-border_thickness:, :] = 255  # Bottom border
    bordered_image[:, :border_thickness] = 255  # Left border
    bordered_image[:, -border_thickness:] = 255  # Right border
    return bordered_image

def preprocess_image(image, kernel_size=(3, 3), blur_ksize=(5, 5), block_size=31, C=10):
    """Preprocess image by applying Gaussian blur and adaptive thresholding."""
    blurred = cv2.GaussianBlur(image, blur_ksize, 0)
    binary_image = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, C
    )
    return binary_image

def find_contours(binary_image):
    """Find contours in the binary image."""
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def detect_corners_from_polygon(contour):
    """Detect corners by approximating the polygon and marking its vertices."""
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    corners = approx.reshape(-1, 2)
    return corners

def cluster_corners(corners, eps=10, min_samples=1):
    """Cluster nearby corners to ensure only one corner per vertex."""
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(corners)
    unique_corners = []
    for label in np.unique(db.labels_):
        cluster_points = corners[db.labels_ == label]
        centroid = np.mean(cluster_points, axis=0).astype(int)
        unique_corners.append(centroid)
    return np.array(unique_corners)

def process_image(image_path, preprocess_params={}):
    """Process a single image, add white border, and detect corners."""
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to read image: {image_path}")
            return

        # Add white border directly on the image
        bordered_image = add_white_border(image, border_thickness=5)

        binary_image = preprocess_image(bordered_image, **preprocess_params)
        contours = find_contours(binary_image)

        contour_image = cv2.cvtColor(bordered_image, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(contour_image, contours, -1, (0, 255, 0), thickness=2)

        plt.figure(figsize=(8, 8))
        plt.imshow(contour_image)
        plt.title('Contours Highlighted with White Border')
        plt.axis('off')
        plt.show()

        for contour in contours:
            corners = detect_corners_from_polygon(contour)
            corners = cluster_corners(corners)

            plt.figure(figsize=(10, 10))
            plt.imshow(bordered_image, cmap='gray')
            plt.title('Detected Corners with White Border')
            plt.axis('off')

            for corner in corners:
                plt.plot(corner[0], corner[1], 'bo', ms=10)

            plt.show()

            print(f"Image: {image_path}")
            print(f"Number of corners detected: {len(corners)}\n")

    except Exception as e:
        print(f"An error occurred while processing '{image_path}': {e}")

# Folder path containing the images
image_folder = "Shapes"

# Check if the folder exists
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Folder '{image_folder}' does not exist.")

# Get all valid image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# If no image files found, exit early
if not image_files:
    print("No valid image files found in the folder.")
else:
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        process_image(image_path)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Function to process each image
def process_image(image_path):
    # Read and convert the image to grayscale
    image_rgb = cv2.imread(image_path)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Thresholding to isolate the cable (adjust thresholds as needed)
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

    # Morphological operations to enhance the bends
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # Rectangular kernel
    dilated = cv2.dilate(thresh, kernel, iterations=2)  # More iterations for stronger dilation
    eroded = cv2.erode(dilated, kernel, iterations=1)

    # Find contours and hierarchy
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # More flexible contour filtering based on hierarchy
    filtered_contours = []
    for i, contour in enumerate(contours):
        # Check if contour is not the outermost (hierarchy level 0)
        if hierarchy is not None and hierarchy[0][i][3] != -1:
            area = cv2.contourArea(contour)
            # Only keep contours within a reasonable area range to filter out noise
            if 10 < area < 610:  # Adjust these thresholds as needed
                filtered_contours.append(contour)

    number_of_bends = len(filtered_contours)

    # Only print and display the image if bends are found
    if number_of_bends > 0:
        print(f'Number of bends found in {os.path.basename(image_path)}: ', number_of_bends)

        # Draw remaining contours on the RGB version of the original image
        contour_image = image_rgb.copy()
        cv2.drawContours(contour_image, filtered_contours, -1, (255, 0, 0), 2)

        # Plot the final image with contours
        plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
        plt.title(f'Image with Contours - {os.path.basename(image_path)}')
        plt.axis('off')
        plt.show()

# Folder containing the images
image_folder = 'Shapes'

# Loop through each image in the folder
for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        image_path = os.path.join(image_folder, filename)
        process_image(image_path)


# Excel Output

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Folder path containing the cropped images
image_folder = "Shapes"

# Excel output file path
excel_output = "corner_detection_results.xlsx"

# Create an empty list to hold the data for the Excel sheet
data = []

# Gaussian blur parameters
gaussian_ksize = (3, 3)
gaussian_sigma = 0  # Default sigma value for Gaussian blur

# Dilation kernel and iterations
dilation_kernel = np.ones((3, 3), np.uint8)
dilation_iterations = 2  # Number of dilation iterations

# Check if the folder exists
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"Folder '{image_folder}' does not exist.")

# Get all valid image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

# If no image files found, exit early
if not image_files:
    print("No valid image files found in the folder.")
else:
    # Loop through all image files in the folder
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)

        try:
            # Read the image in grayscale
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"Failed to read image: {image_file}")
                continue

            # Apply Gaussian blur for noise reduction
            blurred_img = cv2.GaussianBlur(image, gaussian_ksize, gaussian_sigma)

            # Apply dilation to enhance edges
            dilated_img = cv2.dilate(blurred_img, dilation_kernel, dilation_iterations)

            # Shi-Tomasi corner detection
            corners = cv2.goodFeaturesToTrack(
                dilated_img, maxCorners=300, qualityLevel=0.01, minDistance=5
            )

            # Determine the number of corners
            if corners is not None:
                num_corners = len(corners)
                num_bends = max(0, num_corners - 2)  # Avoid negative values

                # Add the data to the list
                data.append({
                    'Image Name': image_file,
                    'Number of Corners': num_corners,
                    'Number of Bends': num_bends
                })

            else:
                # No corners detected, set 0 bends
                data.append({
                    'Image Name': image_file,
                    'Number of Corners': 0,
                    'Number of Bends': 0
                })

        except Exception as e:
            print(f"An error occurred while processing '{image_file}': {e}")

    # Create a DataFrame from the data list
    df = pd.DataFrame(data)

    # Save the DataFrame to an Excel file
    df.to_excel(excel_output, index=False)

    print(f"Results saved to '{excel_output}'")


# Devnotes{ignore}

In [ ]:
'''import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Function to remove patches from an image and display the result
def remove_patches_and_display(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded correctly
    if image is None:
        print(f"Error: Unable to load the image '{image_path}'. Check the file path.")
        return

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Apply adaptive thresholding to create a binary image
    binary_image = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 10
    )

    # Perform morphological operations to refine the binary image
    kernel = np.ones((3, 3), np.uint8)
    binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area to remove small noise
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100]

    # Create a mask filled with white (same size as the original image)
    mask = np.ones_like(binary_image) * 255

    # Draw contours on the mask and fill them with black
    cv2.drawContours(mask, contours, -1, 0, thickness=cv2.FILLED)

    # Create the cleaned image by turning everything outside the contours white
    cleaned_image = cv2.bitwise_or(image, mask)

    # Display the original image, contours, and cleaned image
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))

    # Original image
    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    # Contours highlighted on the original image
    contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), thickness=2)
    ax[1].imshow(contour_image)
    ax[1].set_title('Contours Highlighted')
    ax[1].axis('off')

    # Cleaned image (everything outside the contours is white)
    ax[2].imshow(cleaned_image, cmap='gray')
    ax[2].set_title('Cleaned Image (Outside Contours White)')
    ax[2].axis('off')

    plt.show()

# Path to the folder containing the images
folder_path = "Shapes"

# Iterate over each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)

        # Remove patches and display the result
        remove_patches_and_display(image_path)'''